<a href="https://colab.research.google.com/github/NeuralClassifier/ReusabilityRate_ML/blob/main/CMPUT_663_ReuseRate_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import  LogisticRegression
from sklearn.linear_model import BayesianRidge
from sklearn.linear_model import SGDRegressor
from xgboost import XGBRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import ElasticNet
from sklearn.svm import SVR

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import KFold 
from sklearn.metrics import mean_absolute_error 
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
import statistics

import warnings

In [2]:
!git clone https://github.com/NeuralClassifier/ReusabilityRate_ML

Cloning into 'ReusabilityRate_ML'...
remote: Enumerating objects: 51, done.
remote: Counting objects: 100% (51/51), done.
remote: Compressing objects: 100% (49/49), done.
remote: Total 51 (delta 14), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (51/51), done.


In [14]:
df = pd.read_excel('/content/ReusabilityRate_ML/DataSets/classes.xlsx')
df.drop(['Unnamed: 0'],axis=1,inplace=True)
df

,Project,LongName,Name,LCOM5,NL,NLE,WMC,CBO,CBOI,NII,NOI,RFC,AD,CD,TCD,CLOC,TCLOC,DLOC,PDA,DIT,LLOC,LOC,TLOC,NG,TLLOC,TNA,TNG,TNM,TNOS,TNPM,ReuseRate
0,ch.qos.logback/logback-classic,ch.qos.logback.classic.AsyncAppender,AsyncAppender,2,1,1,5,2,0,0,4,8,0.333333,0.437500,0.437500,14,14,14,1,0,18,29,29,0,18,1,0,4,7,2,6
1,ch.qos.logback/logback-classic,ch.qos.logback.classic.BasicConfigurator,BasicConfigurator,1,0,0,2,5,1,1,3,5,0.333333,0.333333,0.333333,10,10,7,1,1,20,31,31,0,20,0,0,2,14,2,4
2,ch.qos.logback/logback-classic,ch.qos.logback.classic.ClassicConstants,ClassicConstants,0,0,0,0,0,7,0,0,0,0.000000,0.321429,0.321429,9,9,9,0,0,19,35,35,0,19,17,0,0,0,0,6
3,ch.qos.logback/logback-classic,ch.qos.logback.classic.Level,Level,1,1,1,46,0,32,20,0,13,1.000000,0.372549,0.372549,95,95,94,12,0,160,274,274,0,160,24,0,13,68,11,122
4,ch.qos.logback/logback-classic,ch.qos.logback.classic.Logger,Logger,1,3,3,154,5,23,21,11,107,0.074074,0.205696,0.205696,130,130,120,6,0,502,754,754,7,502,10,7,96,248,80,82
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24789,xerces/xercesImpl,org.apache.xml.serialize.SecuritySupport$4,SecuritySupport$4,1,0,0,1,0,0,0,0,1,0.000000,0.000000,0.000000,0,0,0,0,0,5,5,5,0,5,0,0,1,1,1,0
24790,xerces/xercesImpl,org.apache.xml.serialize.SecuritySupport$5,SecuritySupport$5,1,0,0,1,0,0,0,0,1,0.000000,0.000000,0.000000,0,0,0,0,0,5,5,5,0,5,0,0,1,1,1,0
24791,xerces/xercesImpl,org.apache.xml.serialize.SecuritySupport$6,SecuritySupport$6,1,1,1,2,0,0,0,0,1,0.000000,0.000000,0.000000,0,0,0,0,0,11,11,11,0,11,0,0,1,5,1,0
24792,xerces/xercesImpl,org.apache.xml.serialize.SecuritySupport$7,SecuritySupport$7,1,0,0,2,0,0,0,0,1,0.000000,0.000000,0.000000,0,0,0,0,0,5,5,5,0,5,0,0,1,1,1,0


In [5]:
df.iloc[:,3:-1]

,AD,CD,TCD,CLOC,TCLOC,PDA,TPDA,LLOC,LOC,TLOC,NG,TLLOC,TNA,TNG,TNM,TNOS,TNPM,NCL
0,0.000000,0.000000,0.342362,0,4146,0,219,0,0,14288,0,7964,403,217,853,3498,624,0
1,0.000000,0.000000,0.343269,0,4146,0,219,0,0,14250,0,7932,403,217,853,3498,624,0
2,0.000000,0.000000,0.000000,0,0,0,0,32,38,38,0,32,0,0,0,0,0,0
3,0.000000,0.000000,0.343269,0,4146,0,219,0,0,14250,0,7932,403,217,853,3498,624,0
4,0.000000,0.000000,0.343269,0,4146,0,219,0,0,14250,0,7932,403,217,853,3498,624,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2188,0.984127,0.498253,0.498253,713,713,62,62,718,1603,1603,22,718,39,22,66,394,53,10
2189,0.701493,0.476530,0.476530,1137,1137,47,47,1249,2719,2719,17,1249,52,17,91,732,59,10
2190,0.540636,0.482071,0.482071,1909,1909,153,153,2051,4621,4621,94,2051,96,94,312,851,280,15
2191,0.700000,0.225403,0.225403,2543,2543,14,14,8739,12621,12621,34,8739,332,34,290,4838,57,29


In [15]:
warnings.filterwarnings('ignore')
class ReuseTradML:
  def __init__(self,level):
    self.ml_algo_list = ['Polynomial Regression',
                         'MLPRegressor',
                         'BayesianRidge',
                         'SGD Regressor',
                         'XGB Regressor',
                         'Decision Tree Regressor',
                         'ElasticNet',
                         'SVM Regressor']

    self.metrics = ['RMSE',
                    'MAE']
    self.data_columns = None
    self.level=level

    if self.level == 'package':
      target_scaler = MinMaxScaler()
      data = pd.read_excel('/content/ReusabilityRate_ML/DataSets/packages.xlsx')
      data.drop(['Unnamed: 0'],axis=1,inplace=True)
      df = data.iloc[:,3:-1]
      self.data_columns = df.columns
      self.X_np = np.array(df)
      target_scaler.fit(self.X_np)
      self.X = target_scaler.transform(self.X_np)

      self.Y = data.iloc[:,-1]
      self.Y = self.Y.values.reshape(-1, 1)
      target_scaler = MinMaxScaler()
      target_scaler.fit(self.Y)
      self.Y = target_scaler.transform(self.Y)

      self.other = data.iloc[:,:3]

    elif self.level == 'class':
      target_scaler = MinMaxScaler()
      data = pd.read_excel('/content/ReusabilityRate_ML/DataSets/classes.xlsx')
      data.drop(['Unnamed: 0'],axis=1,inplace=True)
      df = data.iloc[:,3:-1]
      self.data_columns = df.columns
      self.X_np = np.array(df)
      target_scaler.fit(self.X_np)
      self.X = target_scaler.transform(self.X_np)

      self.Y = data.iloc[:,-1]
      self.Y = self.Y.values.reshape(-1, 1)
      target_scaler = MinMaxScaler()
      target_scaler.fit(self.Y)
      self.Y = target_scaler.transform(self.Y)

      self.other = data.iloc[:,:3]

    elif self.level == 'project':
      self.X_np = np.array(data.iloc[:,4:])
      self.other = data.iloc[:,:4]

    else:
      print('Error, Please try: class, package, or project. Thanks!')
      self.X=None
      self.Y=None
      self.other=None

  def apply_ml(self):
    kf = KFold(n_splits=10, random_state=None, shuffle=True)
  
    metric = []
    metric_count = 0

    level = []
    model = []

    mean_RMSE = []
    stdv_RMSE = []
    mean_MAE = []
    stdv_MAE = []

    for i in range(self.X.shape[1]):
      #print(self.X[i])
      for j in self.ml_algo_list:
        if j == 'Polynomial Regression':
          
          #temp_data = self.X[i]
          level.append(self.level)
          metric.append(self.data_columns[i])

          cnt=0
          RMSE=[]
          MAE=[]
          poly_reg = PolynomialFeatures(degree=7)
          temp_data = poly_reg.fit_transform(self.X_np[i].reshape(-1, 1))
          pol_reg = LinearRegression()
          model.append('Polynomial Regression')
          warnings.filterwarnings('ignore')

          for train_index, test_index in kf.split(temp_data):
            cnt = cnt+1
            #print('Fold: ',cnt)
            X_train, X_test = temp_data[train_index], temp_data[test_index]
            y_train, y_test = self.Y[train_index], self.Y[test_index]

            pol_reg.fit(X_train,y_train)
            y_pred = pol_reg.predict(X_test)
            RMSE.append(np.sqrt(mean_squared_error(y_test,y_pred)))
            MAE.append(mean_absolute_error(y_test,y_pred))
          
          mean_RMSE.append(statistics.mean(RMSE))
          stdv_RMSE.append(statistics.pstdev(RMSE))
          mean_MAE.append(statistics.mean(MAE))
          stdv_MAE.append(statistics.pstdev(MAE))

        else:
          
          warnings.filterwarnings('ignore')
          #['Polynomial Regression','MLPRegressor','BayesianRidge','SGD Regressor','XGB Regressor','Decision Tree Regressor','ElasticNet','SVM Regressor',]
          if j == 'MLPRegressor':
            reg = MLPRegressor()
            print('Metric: ',self.data_columns[i])
            print('Level: ',self.level)
            print('Model: ',j)
            print()
            print("**************************************************************")
            print()
            model.append(j)

          #if j == 'Logistic Regression':
            #reg = LogisticRegression()
            #print('Metric: ',self.data_columns[i])
            #print('Level: ',self.level)
            #print('Model: ',j)

          if j == 'BayesianRidge':
            reg = BayesianRidge()
            print('Metric: ',self.data_columns[i])
            print('Level: ',self.level)
            print('Model: ',j)
            print()
            print("**************************************************************")
            print()
            model.append(j)

          if j == 'SGD Regressor':
            reg = SGDRegressor()
            print('Metric: ',self.data_columns[i])
            print('Level: ',self.level)
            print('Model: ',j)
            print()
            print("**************************************************************")
            print()
            model.append(j)

          if j == 'XGB Regressor':
            reg = XGBRegressor()
            print('Metric: ',self.data_columns[i])
            print('Level: ',self.level)
            print('Model: ',j)
            print()
            print("**************************************************************")
            print()
            model.append(j)

          if j == 'Decision Tree Regressor':
            reg = DecisionTreeRegressor()
            print('Metric: ',self.data_columns[i])
            print('Level: ',self.level)
            print('Model: ',j)
            print()
            print("**************************************************************")
            print()
            model.append(j)

          if j == 'ElasticNet':
            reg = ElasticNet()
            print('Metric: ',self.data_columns[i])
            print('Level: ',self.level)
            print('Model: ',j)
            print()
            print("**************************************************************")
            print()
            model.append(j)

          if j == 'SVM Regressor':
            reg = SVR()
            print('Metric: ',self.data_columns[i])
            print('Level: ',self.level)
            print('Model: ',j)
            print()
            print("**************************************************************")
            print()
            model.append(j)

          temp_data = self.X[i].reshape(-1, 1)
          level.append(self.level)
          metric.append(self.data_columns[i])

          cnt=0
          RMSE=[]
          MAE=[]
          for train_index, test_index in kf.split(temp_data):
            cnt = cnt+1
            #print('Fold: ',cnt)
            X_train, X_test = temp_data[train_index], temp_data[test_index]
            y_train, y_test = self.Y[train_index], self.Y[test_index]

            reg.fit(X_train,y_train)
            y_pred = reg.predict(X_test)
            RMSE.append(np.sqrt(mean_squared_error(y_test,y_pred)))
            MAE.append(mean_absolute_error(y_test,y_pred))

          mean_RMSE.append(statistics.mean(RMSE))
          stdv_RMSE.append(statistics.pstdev(RMSE))
          mean_MAE.append(statistics.mean(MAE))
          stdv_MAE.append(statistics.pstdev(MAE))
      

       

      #break
    metric = pd.DataFrame(metric,columns=['Metric'])
    level = pd.DataFrame(level,columns=['Level'])
    model = pd.DataFrame(model,columns=['Model'])
    print()

    mean_RMSE = pd.DataFrame(mean_RMSE,columns=['RMSE'])
    stdv_RMSE = pd.DataFrame(stdv_RMSE,columns=['STD RMSE'])
    mean_MAE = pd.DataFrame(mean_MAE,columns=['MAE'])
    stdv_MAE = pd.DataFrame(stdv_MAE,columns=['STD MAE'])

    res_df = pd.concat([metric,level,model,mean_RMSE,stdv_RMSE,mean_MAE,stdv_MAE],axis=1)

    return res_df

In [17]:
warnings.filterwarnings('ignore')
rtm=ReuseTradML(level='class')
res=rtm.apply_ml()
res

Metric:  LCOM5
Level:  class
Model:  MLPRegressor

**************************************************************

Metric:  LCOM5
Level:  class
Model:  BayesianRidge

**************************************************************

Metric:  LCOM5
Level:  class
Model:  SGD Regressor

**************************************************************

Metric:  LCOM5
Level:  class
Model:  XGB Regressor

**************************************************************

[14:28:50] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:28:50] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:28:50] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:28:50] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:28:50] WARNING: /workspac

,Metric,Level,Model,RMSE,STD RMSE,MAE,STD MAE
0,LCOM5,class,Polynomial Regression,0.009643,0.007003,0.007600,0.005483
1,LCOM5,class,MLPRegressor,0.028671,0.016375,0.026074,0.014129
2,LCOM5,class,BayesianRidge,0.008408,0.006203,0.007088,0.004094
3,LCOM5,class,SGD Regressor,0.006753,0.007981,0.005309,0.005629
4,LCOM5,class,XGB Regressor,0.011022,0.006867,0.007839,0.004356
...,...,...,...,...,...,...,...
211,TNPM,class,SGD Regressor,0.007336,0.007549,0.005328,0.004568
212,TNPM,class,XGB Regressor,0.011120,0.007308,0.007887,0.005114
213,TNPM,class,Decision Tree Regressor,0.009274,0.008335,0.006852,0.006062
214,TNPM,class,ElasticNet,0.008602,0.006806,0.007438,0.005183


In [18]:
res.to_excel('/content/classRes_TraditionalModels.xlsx')